In [ ]:
import os
import sys
app_path = os.path.abspath('..')
sys.path.insert(0, app_path)

from datetime import datetime, UTC

from langchain import PromptTemplate
from langchain_ollama import ChatOllama

from app.chroma_client import get_embeddings, get_client

In [ ]:
channel_name = 'go_to_vilnius'
channel_id = -1133953167

In [ ]:
# ru_model_name = "cointegrated/rubert-tiny2"
ru_model_name = "intfloat/multilingual-e5-large"
embeddings = get_embeddings(ru_model_name)

chroma_client_from_telegram = get_client(f'telegram_{channel_name}', embeddings)

In [ ]:
llm_qwen3_8b = ChatOllama(
    model="qwen3:8b",
)

In [ ]:
user_query = "Выведи самые токсичные сообщения в чате."
user_query = "Выведи самые позитивные сообщения о Литве и Вильнюсе в чате."
# user_query = "Можно ли компенсировать отель или аквапарк через велнес/БТА/бта/BTA?"
# user_query = "Как получить детские деньги или компенсацию по уходу за ребенком."

In [ ]:
related_docs = chroma_client_from_telegram.similarity_search_with_relevance_scores(user_query, k=15)
related_docs

In [ ]:
filtered_related_docs = filter(lambda doc_score: doc_score[-1] > 0.3, related_docs)

t_link_base = "https://web.telegram.org/k/#?tgaddr=tg%3A%2F%2Fprivatepost%3Fchannel%3D1133953167%26post%3D"

print("\n\n---\n\n".join(f"""
{datetime.fromtimestamp(doc.metadata['date'], UTC)} {t_link_base}{doc.metadata['id']} - {doc.page_content}
""" for doc, _score in filtered_related_docs))

In [ ]:
filtered_related_docs = filter(lambda doc_score: doc_score[-1] > 0.3, related_docs)

context = "\n\n---\n\n".join(f"{datetime.fromtimestamp(doc.metadata['date'], UTC)} - {doc.page_content}" for doc, _score in filtered_related_docs)

In [ ]:
ru_telegram_prompt = PromptTemplate(
    input_variables=["context", "user_query"],
    template="""
    Ты полезный AI ассистент, который отвечает на вопросы пользователя на основе контекста.
    Контекст это релевантные вопросу сообщения из телеграм чата.

    Контекст:
    {context}

    Вопрос пользователя:
    {user_query}

    Если в контексте недостаточно информации, чтобы ответить на вопрос пользователя, то скажи, что недостаточно информации.

    Ответ:
    """,
)

hyde_chain_r1_8b = ru_telegram_prompt | llm_qwen3_8b
llm_response = hyde_chain_r1_8b.invoke({"user_query": user_query, "context": context}).content
print(llm_response)